February 21, 2016: initial sklearn development

---

There isn't that much to port over. Mostly just the main classes, since that cannot be translated. We'll have opportunities to use the new PyCall in some cases, at least.

# sklearn

In [ ]:
class MajorityClassifier(BaseEstimator, ClassifierMixin):
...     """Predicts the majority class of its training data."""
...     def __init__(self):
...         pass
...
...     def fit(self, X, y):
...         self.classes_, indices = np.unique(["foo", "bar", "foo"],
...                                            return_inverse=True)
...         self.majority_ = np.argmax(np.bincount(indices))
...         return self
...
...     def predict(self, X):
...         return np.repeat(self.classes_[self.majority_], len(X))


In [1]:
using PyCall

In [2]:
@pyimport sklearn.base as skbase


In [3]:
using PyCall
@pyimport numpy as np

type MajorityClassifier
    classes_
    majority_
    MajorityClassifier() = new()
end

@pydef type MajorityClassifier
    fit(self, X, y) = fit!(self, X, y)
    predict(self, X) = predict(self, X)
end

function fit!(mc::MajorityClassifier, X, y)
    mc.classes_, indices = np.unique(y, return_inverse=true)
    mc.majority_ = np.argmax(np.bincount(indices)) + 1
    return mc
end

function predict(mc::MajorityClassifier, X)
    fill(mc.classes_[mc.majority_], size(X, 1))
end

predict (generic function with 1 method)

In [5]:
mc_py = PyObject(MajorityClassifier())
mc_py[:fit]([101, 102, 101]'', [true, false, false])
mc_py[:predict]([101, 102, 101]'')

3-element Array{Bool,1}:
 false
 false
 false

In [6]:
mc = convert(PyAny, mc_py)

MajorityClassifier(Bool[false,true],1)

In [33]:
mc.classes_

2-element Array{Int64,1}:
 101
 102

In [12]:
@pyimport sklearn.utils as sku

In [1]:
using PyCall

unshift!(PyVector(pyimport("sys")["path"]), "..");
@pyimport Shouting

@pyimport sklearn.utils as sku
@pyimport sklearn.cross_validation as skcross
@pyimport sklearn.datasets as skdata
@pyimport sklearn.base as skbase

iris = skdata.load_iris()

XX, yy = sku.shuffle(iris["data"][:,:], iris["target"][:])

@pyimport numpy as np
@pyimport sklearn.metrics as skmetrics

type MajorityClassifier
    classes_
    majority_
    MajorityClassifier() = new(nothing, nothing)
end

@pydef type MajorityClassifier # <: skbase.BaseEstimator #Shouting.Shouter
    fit(self, X, y) = fit!(self, X, y)
    predict(self, X) = predict(self, X)
    score(self, X, y; sample_weight=nothing) = score(self, X, y; sample_weight=sample_weight)
    get_params(self; deep=false) = Dict()
end

function fit!(mc::MajorityClassifier, X, y)
    mc.classes_, indices = np.unique(y, return_inverse=true)
    mc.majority_ = np.argmax(np.bincount(indices))
    return mc
end

function predict(mc::MajorityClassifier, X)
    fill(mc.classes_[mc.majority_], size(X, 1))
end

function score(mc::MajorityClassifier, X, y; sample_weight=nothing)
    return skmetrics.accuracy_score(y, predict(mc, X), sample_weight=sample_weight)
end


#clf = svm.SVC(kernel='linear', C=1)
clf = MajorityClassifier()
#skcross.cross_val_score(clf, XX, yy, cv=5, )

MajorityClassifier(nothing,nothing)

In [2]:
PyObject(clf)[:fit](XX, yy)

MajorityClassifier([0,1,2],0)

In [ ]:
skbase.clone(clf)